
# Introduction à l’IA Générative — **Phase 1 : Basic Prompt Structure**  
**Modèle** : *Llama‑3.2‑1B (Instruct)* via **Ollama**  
**Objectif** : Comprendre et appliquer la structure d’un *prompt* de base pour obtenir des réponses plus pertinentes.

> Cette leçon se concentre uniquement sur la **structure du prompt**. Les rôles (system/user/assistant), la séparation data/instructions, le formatage, les few‑shots et les prompts complexes viendront dans les phases suivantes.



## Prérequis
- **Ollama** installé et lancé en local (serveur actif).
- Le package Python `ollama` (client) pour appeler l'API locale.
- Modèle : `llama3.2:1b` (compatible avec les machines modestes).

> Si Ollama n’est pas lancé, démarrez-le d’abord. Ensuite, exécutez les cellules ci‑dessous.


In [ ]:

# (Optionnel) Installer/mettre à jour le client Python Ollama
# Décommentez si besoin :
# !pip install -q --upgrade ollama


In [1]:

import ollama

def assert_ollama_ready():
    try:
        _ = ollama.list()
        print("✅ Ollama est accessible en local.")
    except Exception as e:
        raise SystemExit(
            "❌ Impossible de contacter Ollama. Assurez-vous que le serveur tourne (ex: 'ollama serve').\n"
            f"Détails: {e}"
        )

assert_ollama_ready()


✅ Ollama est accessible en local.


In [ ]:
try:
    print("⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...")
    ollama.pull('llama3.2:1b')
    print("✅ Modèle prêt.")
except Exception as e:
    print("ℹ️ Si le pull échoue, vérifiez le tag de modèle (ex: 'llama3.2:1b' ou 'llama3.2:1b-instruct').")
    raise



## Exemple minimal : un prompt simple
Le prompt est juste une **instruction** claire. On commence par la version la plus simple possible.


# Exemple Via Huggingface

In [ ]:
!pip install transformers accelerate torch

In [ ]:
from huggingface_hub import login
login(token="hf_huggingfaceapi")

c:\Users\salah\anaconda3\envs\ragllm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_id = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_id)  
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")


prompt = "Définis l'intelligence artificielle en une phrase."

# Tokenisation
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Génération
outputs = model.generate(**inputs, max_new_tokens=100)

# Décodage
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Réponse :", response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Réponse : Définis l'intelligence artificielle en une phrase. La nouvelle Intelligence Artificielle, ou IA, est le résultat de la fusion entre la science et l'art. Les plus grands scientifiques et ingénieurs du monde entier se sont unis pour créer l'IA. L'IA peut être vue comme un outil de révolution qui permettra à la société de prendre un pas en avant dans le domaine de l'intelligence artificielle. L'IA peut être utilisée pour créer de nouveaux services,


# Exemple via API :

In [ ]:
# Chargement de la clé api 
import json
from openai import OpenAI

with open("config.json") as f:
    config = json.load(f)

api_key = config["OPENAI_API_KEY"]
client = OpenAI(api_key=api_key)

In [11]:
prompt = "Définis l'intelligence artificielle en une phrase."

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

print("Réponse :", response.choices[0].message.content)

Réponse : L'intelligence artificielle est un domaine de l'informatique qui vise à créer des systèmes capables d'accomplir des tâches nécessitant normalement l'intelligence humaine, telles que la reconnaissance de la parole, la prise de décisions, et la compréhension du langage naturel.



## Anatomie d’un bon *prompt* (structure de base)
Un *prompt* efficace comprend généralement :

1. **Tâche/objectif clair** — ce que vous voulez que le modèle fasse.
2. **Contexte** (facultatif mais utile) — ce que le modèle doit savoir pour mieux répondre.
3. **Contraintes** — longueur, style, ton, niveau de détail, interdits, etc.
4. **Format de sortie** (basique ici) — phrase, liste, étapes numérotées, etc.

> **Règle d’or** : *clair, concret, contraint*. Évitez les formulations vagues.


In [ ]:

def ask(prompt: str, temperature: float = 0.2):
    
    
    r = ollama.chat(
        model='llama3.2:1b',
        messages=[{'role': 'user', 'content': prompt}],
        options={'temperature': temperature}
    )
    return r['message']['content']


print(ask("En une phrase, explique la différence entre IA faible et IA forte."))



## Démo : flou vs. précis
Comparons un prompt **vague** à un prompt **précis** avec contraintes.


In [ ]:

prompt_vague = "Parle de l'IA."
prompt_precis = (
    "Explique brièvement ce qu’est l’intelligence artificielle en **3 points numérotés** : "
    "1) définition simple, 2) un exemple concret du quotidien, 3) une limite actuelle. "
    "Utilise des phrases courtes."
)

print("— Prompt vague —")
print(prompt_vague)
print("\nRéponse :\n", ask(prompt_vague))

print("\n\n— Prompt précis —")
print(prompt_precis)
print("\nRéponse :\n", ask(prompt_precis))



## Modèle (template) de prompt de base
Utilisez ce canevas et remplacez `{{...}}` :

```
TÂCHE : {{ce que je veux que le modèle fasse en une phrase}}

CONTEXTE (optionnel) : {{faits, hypothèses, public visé, contraintes de domaine}}

CONTRAINTES : 
- Longueur : {{ex. 100–150 mots / 3 points / une phrase}}
- Style/ton : {{clair, pédagogique, professionnel, grand public…}}
- Interdits : {{jargon inutile, spéculation, etc.}}

FORMAT DE SORTIE : {{phrase, liste à puces, étapes numérotées, etc.}}
```

> Astuce : commencez simple, ajoutez des contraintes seulement si nécessaire.
